In [3]:
%cd ..
%pwd  # To verify the current working directory

c:\Users\happy\Documents\Projects\askgrowbuddy


c:\Users\happy\Documents\Projects\askgrowbuddy\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'c:\\Users\\happy\\Documents\\Projects\\askgrowbuddy'

In [4]:
from src.soil_test_analyst import SoilTestAnalyst
# from src.soil_test_models import soil_report_instance

resource module not available on Windows


In [1]:
mehlic_upload = r'C:\Users\happy\Documents\Projects\askgrowbuddy\Margaret Johnson-Soil-20240911-179093.pdf'
paste_upload = r'C:\Users\happy\Documents\Projects\askgrowbuddy\Margaret Johnson-Saturated Paste-20240911-179093.pdf'
# m3_report, sp_report = SoilTestAnalyst.load_reports(mehlic_upload, paste_upload)
# soil_report_instance.set_reports(m3_report, sp_report)

In [ ]:
analyst = SoilTestAnalyst(mehlic_upload, paste_upload)


In [4]:
soil_property_name = "ph"
value = 6.5
optimal_range = "6.0-7.0"

In [5]:
# This section requires this code.
import nest_asyncio
nest_asyncio.apply()

In [6]:

retriever_prompt = f"The  {soil_property_name} at value {value}; optimal range is {optimal_range}. Includes actionable advice on adjustments if current value is outside optimal conditions."
nodes_with_score = analyst.retriever.retrieve(retriever_prompt)

In [26]:
retriever_prompt= "The ph is at 6.9.  What is the optimal range for growing Cannabis?"

In [28]:
nodes_with_score = analyst.retriever.retrieve(retriever_prompt)

In [29]:
dir(nodes_with_score[0])
nodes_with_score[0].metadata


{'source': '2024-09-11 Preparation Document for Mehlic-3 Analysis.md',
 'path': 'G:\\My Drive\\Audios_To_Knowledge\\knowledge\\AskGrowBuddy\\AskGrowBuddy\\Knowledge\\soil_test_knowlege\\soil test comments\\2024-09-11\\2024-09-11 Preparation Document for Mehlic-3 Analysis.md',
 'created': 1726950458.577,
 'last_modified': 1726606704.565,
 'last_accessed': 1726950458.577,
 'Header_1': 'Mehlic-3',
 'Header_2': 'Trace Elements',
 'Header_3': 'Aluminum (ppm) is 30',
 'retriever_source': 'bm25'}

In [34]:
import gradio as gr
import traceback
import logging
import os

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

def create_node_viewer(nodes_with_score, retriever_prompt):
    def format_metadata(metadata):
        formatted = "Metadata:\n"
        for key, value in metadata.items():
            if key == 'path':
                value = os.path.basename(value)
            formatted += f"- {key}: {value}\n"
        return formatted

    def show_node(index):
        try:
            logger.debug(f"Showing node at index {index}")
            node_with_score = nodes_with_score[index]
            formatted_metadata = format_metadata(node_with_score.metadata)
            filename = os.path.basename(node_with_score.metadata['path'])
            return (
                node_with_score.get_content(),
                f"Retriever Source: {node_with_score.metadata['retriever_source']}",
                f"Content Source: {filename}",
                f"Score: {node_with_score.get_score():.4f}",
                formatted_metadata,
                f"Node {index + 1} of {len(nodes_with_score)}",
                index
            )
        except Exception as e:
            logger.error(f"Error displaying node: {str(e)}")
            logger.error(traceback.format_exc())
            return "Error displaying node", "Error", "Error", "Error", "Error formatting metadata", f"Error at index {index}", index

    def navigate(direction, current_index):
        new_index = (int(current_index) + direction) % len(nodes_with_score)
        return show_node(new_index)

    with gr.Blocks() as iface:
        gr.Markdown(f"# {retriever_prompt}")

        current_index = gr.State(0)

        with gr.Row():
            prev_btn = gr.Button("Previous")
            next_btn = gr.Button("Next")

        node_text = gr.Textbox(label="Node Content", lines=10)
        retriever_source = gr.Markdown()
        content_source = gr.Markdown()
        score = gr.Markdown()
        metadata = gr.Markdown()
        index_display = gr.Markdown()

        prev_btn.click(navigate, inputs=[gr.Number(-1), current_index],
                       outputs=[node_text, retriever_source, content_source, score, metadata, index_display, current_index])
        next_btn.click(navigate, inputs=[gr.Number(1), current_index],
                       outputs=[node_text, retriever_source, content_source, score, metadata, index_display, current_index])

        # Initialize with the first node
        iface.load(show_node, inputs=[gr.Number(0)],
                   outputs=[node_text, retriever_source, content_source, score, metadata, index_display, current_index])

    return iface

# Assuming nodes_with_score and retriever_prompt are already defined
retriever_prompt = "Cannabis cultivation guidance for ph at value 6.5; optimal range is 6.0-7.0. Includes actionable advice on adjustments if current value is outside optimal conditions."

# Create and launch the interface
iface = create_node_viewer(nodes_with_score, retriever_prompt)
iface.launch(inline=True)

* Running on local URL:  http://127.0.0.1:7875

To create a public link, set `share=True` in `launch()`.


In [ ]:
from IPython.display import display, clear_output, HTML

def create_node_viewer(nodes_with_score, retriever_prompt):
    current_index = 0

    def show_prompt_and_node(index):
        clear_output(wait=True)

        # Display the retriever prompt
        display(HTML(f"<h3>Retriever Prompt:</h3><p>{retriever_prompt}</p><hr>"))

        node_with_score = nodes_with_score[index]
        print(f"Node {index + 1} of {len(nodes_with_score)}")
        print(f"Score: {node_with_score.get_score():.4f}")
        print("\nContent:")
        print(node_with_score.get_content())
        print("\nMetadata:")
        print(node_with_score.metadata)
        print("\nEnter 'n' for next, 'p' for previous, or 'q' to quit.")

    while True:
        show_prompt_and_node(current_index)
        user_input = input().lower()

        if user_input == 'n':
            current_index = (current_index + 1) % len(nodes_with_score)
        elif user_input == 'p':
            current_index = (current_index - 1) % len(nodes_with_score)
        elif user_input == 'q':
            break
        else:
            print("Invalid input. Please enter 'n', 'p', or 'q'.")

# Assuming nodes_with_score and retriever_prompt are already defined
create_node_viewer(nodes_with_score, retriever_prompt)